In [3]:
import os
os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_key

'73bdec39-1f93-47fc-bd2f-f02883d7be83'

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import PyPDF2
import unidecode



# Data vectorization


## convert pdf to .txt


In [26]:
def normalize_filename(filename):
    filename = unidecode.unidecode(filename)
    filename = filename.replace(" ", "_")
    return filename

def pdf_to_text(pdf_file, txt_file):
    with open(pdf_file, 'rb') as pdf:
        pdf_reader = PyPDF2.PdfReader(pdf)
        text = ""

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    with open(txt_file, 'w', encoding='utf-8') as txt:
        txt.write(text)

def convert_pdfs_in_folder(pdf_path, txt_path):
    os.makedirs(txt_path, exist_ok=True)
    for filename in os.listdir(pdf_path):
        if filename.endswith('.pdf'):
            nomalized_filename = normalize_filename(filename)
            pdf_file = os.path.join(pdf_path, filename)
            txt_file = os.path.join(txt_path, nomalized_filename.replace('.pdf', '.txt'))
            pdf_to_text(pdf_file, txt_file)
            print(f'Đã chuyển đổi {pdf_file} thành {txt_file}')

In [12]:
pdf_file = 'data/pdf_files/Công nghệ chế biến lâm sản.pdf'  # Đường dẫn đến tệp PDF
txt_file = 'data/txt_files/output.txt' # Đường dẫn đến tệp .txt bạn muốn lưu

pdf_to_text(pdf_file, txt_file)

In [27]:
pdf_path = 'data/pdf_files'
txt_path = 'data/txt_files'
convert_pdfs_in_folder(pdf_path, txt_path)

Đã chuyển đổi data/pdf_files\25_NganhCongNgheVatLieu.pdf thành data/txt_files\25_NganhCongNgheVatLieu.txt
Đã chuyển đổi data/pdf_files\CKM_NganhCNKTCoDienTu.pdf thành data/txt_files\CKM_NganhCNKTCoDienTu.txt
Đã chuyển đổi data/pdf_files\Công nghệ chế biến lâm sản.pdf thành data/txt_files\Cong_nghe_che_bien_lam_san.txt
Đã chuyển đổi data/pdf_files\Công nghệ chế tạo máy.pdf thành data/txt_files\Cong_nghe_che_tao_may.txt
Đã chuyển đổi data/pdf_files\Công nghệ in.pdf thành data/txt_files\Cong_nghe_in.txt
Đã chuyển đổi data/pdf_files\Công nghệ kỹ thuật công trình xây dựng.pdf thành data/txt_files\Cong_nghe_ky_thuat_cong_trinh_xay_dung.txt
Đã chuyển đổi data/pdf_files\Công nghệ kỹ thuật cơ khí.pdf thành data/txt_files\Cong_nghe_ky_thuat_co_khi.txt
Đã chuyển đổi data/pdf_files\Công nghệ Kỹ thuật Hóa học.pdf thành data/txt_files\Cong_nghe_Ky_thuat_Hoa_hoc.txt
Đã chuyển đổi data/pdf_files\Công nghệ kỹ thuật máy tính.pdf thành data/txt_files\Cong_nghe_ky_thuat_may_tinh.txt
Đã chuyển đổi data/pdf


## Load data


In [29]:
class UTF8TextLoader(TextLoader):
    def __init__(self, file_path):
        super().__init__(file_path, encoding='utf-8')

pdf_path = 'data/txt_files'
loader = DirectoryLoader(path=pdf_path, glob='*.txt', loader_cls=UTF8TextLoader)
documents = loader.load()

## Data split

In [31]:
separators = [
    "\n\n",
    "\n",
    " ",
    ".",
    ",",
    "\u200b",  # Zero-width space
    "\uff0c",  # Fullwidth comma
    "\u3001",  # Ideographic comma
    "\uff0e",  # Fullwidth full stop
    "\u3002",  # Ideographic full stop
    "",
]
chunk_size = 500
chunk_overlap = 50

def word_count(text):
    return len(text.split())

In [32]:
#splitter
document_splitter = RecursiveCharacterTextSplitter(
    separators = separators,
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = word_count,
    is_separator_regex = False
)
#embedding
#vector_store

In [33]:
chunks = document_splitter.split_documents(documents)
chunks

[Document(metadata={'source': 'data\\txt_files\\25_NganhCongNgheVatLieu.txt'}, page_content='BỘ GIÁO DỤC VÀ ĐÀO TẠO  \nTRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT  \nTHÀNH PHỐ HỒ CHÍ MINH  \n \n \n \n \n \nCHƯƠNG TRÌNH GIÁO DỤC ĐẠI HỌC  \nNGÀNH CÔNG NGHỆ VẬT LIỆU  \n150 Tín chỉ  \n \n Tên chương trình: CÔNG NGHỆ VẬT LIỆU  \n Ngành đào tạo : CÔNG NGHỆ VẬT LIỆU     \nTên tiếng Anh: MATERIALS TECHNOLOGY  \nTrình độ đào tạo: ĐẠI HỌC  \nMã ngành : 7510402  \n Hình thức đào tạo : CHÍNH QUI  \n       (Ban hành tại Quyết định số  973/QĐ -ĐHSPKT, ngày 19/03/2021 của Hiệu \ntrưởng trường Đại học Sư phạm Kỹ thuật Thành phố Hồ Chí Minh)  \n Áp dụng từ khóa tuyển sinh 20 23 \n \n \n \n \n \n \nTP. Hồ Chí Minh  \n 2 \n BỘ GIÁO DỤC VÀ ĐÀO TẠO  \nTRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT  \nTHÀNH PHỐ HỒ CHÍ MINH  \n CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM  \nĐộc lập - Tự do - Hạnh phúc  \nCHƯƠNG TRÌNH GIÁO DỤC ĐẠI HỌC  \n  \nTên chương trình: CÔNG NGHỆ VẬT LIỆU  \n Trình độ đào tạo:  Đại học  \n Ngành đào tạo : CÔNG NGHỆ VẬT LI

## Embedding model

In [35]:
embedding_model = HuggingFaceEmbeddings()

In [38]:
embedded_txt = embedding_model.embed_query('Học phần pháp luật đại cương trang bị cho sinh viên điều gì?')
embedded_txt

[0.037909336388111115,
 -0.05401851609349251,
 0.0009662903612479568,
 0.020404640585184097,
 0.011347148567438126,
 0.02341471053659916,
 -0.05361638590693474,
 0.006261014845222235,
 0.008326255716383457,
 -0.01616784743964672,
 -0.04294917359948158,
 0.0017678086878731847,
 0.04487469419836998,
 -0.014933091588318348,
 -0.01982441358268261,
 -0.035548996180295944,
 0.020211827009916306,
 -0.0013785240007564425,
 0.014603815972805023,
 -0.05087968334555626,
 -0.023231368511915207,
 0.03623106703162193,
 0.023890022188425064,
 0.07251973450183868,
 0.06349549442529678,
 -0.04291483387351036,
 0.0019599187653511763,
 -0.011090544052422047,
 0.000805511197540909,
 -0.024050313979387283,
 0.03006887435913086,
 -0.005565640516579151,
 -0.009172294288873672,
 -0.04378516599535942,
 2.128636651832494e-06,
 -0.04193031042814255,
 0.0068909768015146255,
 -0.033436745405197144,
 -0.0588444247841835,
 0.016701992601156235,
 0.0270204059779644,
 -0.028275595977902412,
 -0.016245657578110695,
 -0

## Vector store


In [36]:
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index('docs-rag-chatbot')

In [37]:
db = PineconeVectorStore.from_documents(documents=chunks, embedding=embedding_model, index_name="docs-rag-chatbot", 
                                            namespace="docs-ute-store" )

## Query vector database



### Create vectordb class

In [1]:
class VectorDB:
    def __init__(self):
        self.embedding = HuggingFaceEmbeddings()
        self.db = self.__build_db__()

    def __build_db__(self):
        db = PineconeVectorStore.from_existing_index(
            index_name="docs-rag-chatbot",
            namespace="docs-ute-store",
            embedding=self.embedding
        )
        return db
    
    def get_retriever(self,
                      search_type: str = 'similarity', 
                      search_kwargs: dict = {'k': 10}):
        retriever = self.db.as_retriever(search_type=search_type, search_kwargs=search_kwargs)
        return retriever


